In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'prepare/mesolitica-tpu.json'
b2_application_key_id = os.environ['b2_application_key_id']
b2_application_key = os.environ['b2_application_key']

In [2]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-tpu-general')

In [3]:
best = '1100000'
directory = 't5-super-super-tiny-true-case'
!rm -rf output out {directory}
!mkdir {directory}

In [4]:
model = best

blob = bucket.blob(f'{directory}/model.ckpt-{model}.data-00000-of-00002')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.data-00000-of-00002')

blob = bucket.blob(f'{directory}/model.ckpt-{model}.data-00001-of-00002')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.data-00001-of-00002')

blob = bucket.blob(f'{directory}/model.ckpt-{model}.index')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.index')

blob = bucket.blob(f'{directory}/model.ckpt-{model}.meta')
blob.download_to_filename(f'{directory}/model.ckpt-{model}.meta')

blob = bucket.blob(f'{directory}/checkpoint')
blob.download_to_filename(f'{directory}/checkpoint')

blob = bucket.blob(f'{directory}/operative_config.gin')
blob.download_to_filename(f'{directory}/operative_config.gin')

In [5]:
with open(f'{directory}/checkpoint', 'w') as fopen:
    fopen.write(f'model_checkpoint_path: "model.ckpt-{model}"')

In [6]:
from b2sdk.v1 import *
info = InMemoryAccountInfo()
b2_api = B2Api(info)
application_key_id = b2_application_key_id
application_key = b2_application_key
b2_api.authorize_account("production", application_key_id, application_key)
file_info = {'how': 'good-file'}
b2_bucket = b2_api.get_bucket_by_name('malaya-model')

In [7]:
tar = 't5-super-super-tiny-true-case-2021-09-10.tar.gz'
os.system(f'tar -czvf {tar} {directory}')
outPutname = f'finetuned/{tar}'
b2_bucket.upload_local_file(
    local_file=tar,
    file_name=outPutname,
    file_infos=file_info,
)

In [8]:
os.system(f'rm {tar}')

0

In [9]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

In [10]:
model = t5.models.MtfModel(
    model_dir=directory,
    tpu=None,
    tpu_topology=None,
    model_parallelism=1,
    batch_size=1,
    sequence_length={"inputs": 256, "targets": 256},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["cpu:0"]
)

In [11]:
!rm -rf output/*

In [12]:
import gin

from t5.data import sentencepiece_vocabulary

DEFAULT_SPM_PATH = 'prepare/sp10m.cased.ms-en.model'
DEFAULT_EXTRA_IDS = 100
model_dir = directory

def get_default_vocabulary():
    return sentencepiece_vocabulary.SentencePieceVocabulary(
      DEFAULT_SPM_PATH, DEFAULT_EXTRA_IDS)

with gin.unlock_config():
    gin.parse_config_file(t5.models.mtf_model._operative_config_path(model_dir))
    gin.bind_parameter("Bitransformer.decode.beam_size", 1)
    gin.bind_parameter("Bitransformer.decode.temperature", 0)
    gin.bind_parameter("utils.get_variable_dtype.slice_dtype", "float32")
    gin.bind_parameter(
        "utils.get_variable_dtype.activation_dtype", "float32")
    
vocabulary = t5.data.SentencePieceVocabulary(DEFAULT_SPM_PATH)
estimator = model.estimator(vocabulary, disable_tpu=True)

INFO:tensorflow:Using config: {'_model_dir': 't5-super-super-tiny-true-case', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': None, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, per_host_input_for_training=4, tpu_job_name=None, initial_infeed_sleep_secs=None, input_partition_dims=None,

In [13]:
import os

checkpoint_step = t5.models.mtf_model._get_latest_checkpoint_from_dir(model_dir)
model_ckpt = "model.ckpt-" + str(checkpoint_step)
checkpoint_path = os.path.join(model_dir, model_ckpt)
checkpoint_step, model_ckpt, checkpoint_path

(1100000,
 'model.ckpt-1100000',
 't5-super-super-tiny-true-case/model.ckpt-1100000')

In [14]:
from mesh_tensorflow.transformer import dataset as transformer_dataset

def serving_input_fn():
    inputs = tf.placeholder(
            dtype=tf.string,
            shape=[None],
            name="inputs")

    batch_size = tf.shape(inputs)[0]
    padded_inputs = tf.pad(inputs, [(0, tf.mod(-tf.size(inputs), batch_size))])
    dataset = tf.data.Dataset.from_tensor_slices(padded_inputs)
    dataset = dataset.map(lambda x: {"inputs": x})
    dataset = transformer_dataset.encode_all_features(dataset, vocabulary)
    dataset = transformer_dataset.pack_or_pad(
        dataset=dataset,
        length=model._sequence_length,
        pack=False,
        feature_keys=["inputs"]
    )
    dataset = dataset.batch(tf.cast(batch_size, tf.int64))
    features = tf.data.experimental.get_single_element(dataset)
    return tf.estimator.export.ServingInputReceiver(
        features=features, receiver_tensors=inputs)

out = estimator.export_saved_model('output', serving_input_fn, checkpoint_path=checkpoint_path)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:feature inputs : Tensor("Reshape:0", shape=(1, 256), dtype=int32)
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:

INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/k                  size 49152        slice_size 49152        Shape[d_model=128, heads=384]                               
INFO:tensorflow:Variable decoder/block_000/layer_000/SelfAttention/o                  size 49152        slice_size 49152        Shape[heads=384, d_model=128] 

INFO:tensorflow:Variable encoder/block_000/layer_001/DenseReluDense/wi/kernel         size 65536        slice_size 65536        Shape[d_model=128, d_ff=512]                                
INFO:tensorflow:Variable encoder/block_000/layer_001/DenseReluDense/wo/kernel         size 65536        slice_size 65536        Shape[d_ff=512, d_model=128]                                
INFO:tensorflow:Variable encoder/block_000/layer_001/layer_norm/scale                 size 128          slice_size 128          Shape[d_model=128]                                          
INFO:tensorflow:Variable encoder/block_001/layer_000/SelfAttention/k                  size 49152        slice_size 49152        Shape[d_model=128, heads=384]                               
INFO:tensorflow:Variable encoder/block_001/layer_000/SelfAttention/o                  size 49152        slice_size 49152        Shape[heads=384, d_model=128]                               
INFO:tensorflow:Variable encoder/block_001/layer_000/Se

In [15]:
config = tf.ConfigProto()
config.allow_soft_placement = True
sess = tf.Session(config = config)
meta_graph_def = tf.saved_model.loader.load(
        sess,
        [tf.saved_model.tag_constants.SERVING],
        out)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from output/1633111550/variables/variables


In [16]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'super-super-tiny-true-case/model.ckpt')

'super-super-tiny-true-case/model.ckpt'

In [17]:
strings = [
    n.name
    for n in tf.get_default_graph().as_graph_def().node
    if ('encoder' in n.op
    or 'decoder' in n.name
    or 'shared' in n.name
    or 'inputs' in n.name
    or 'output' in n.name
    or 'SentenceTokenizer' in n.name
    or 'self/Softmax' in n.name)
    and 'adam' not in n.name
    and 'Assign' not in n.name
]

In [18]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names,
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [19]:
freeze_graph('super-super-tiny-true-case', strings)

INFO:tensorflow:Restoring parameters from super-super-tiny-true-case/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 76 variables.
INFO:tensorflow:Converted 76 variables to const ops.
2616 ops in the final graph.


In [20]:
import struct

unknown = b'\xff\xff\xff\xff'

def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    for node in graph_def.node:
        
        if node.op == 'RefSwitch':
          node.op = 'Switch'
          for index in xrange(len(node.input)):
            if 'moving_' in node.input[index]:
              node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
          node.op = 'Sub'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'AssignAdd':
          node.op = 'Add'
          if 'use_locking' in node.attr: del node.attr['use_locking']
        elif node.op == 'Assign':
          node.op = 'Identity'
          if 'use_locking' in node.attr: del node.attr['use_locking']
          if 'validate_shape' in node.attr: del node.attr['validate_shape']
          if len(node.input) == 2:
            node.input[0] = node.input[1]
            del node.input[1]
            
        if 'Reshape/shape' in node.name or 'Reshape_1/shape' in node.name:
            b = node.attr['value'].tensor.tensor_content
            arr_int = [int.from_bytes(b[i:i + 4], 'little') for i in range(0, len(b), 4)]
            if len(arr_int):
                arr_byte = [unknown] + [struct.pack('<i', i) for i in arr_int[1:]]
                arr_byte = b''.join(arr_byte)
                node.attr['value'].tensor.tensor_content = arr_byte
            
            if len(node.attr['value'].tensor.int_val):
                node.attr['value'].tensor.int_val[0] = -1
    
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [21]:
g = load_graph('super-super-tiny-true-case/frozen_model.pb')

In [22]:
i = g.get_tensor_by_name('import/inputs:0')
o = g.get_tensor_by_name('import/SelectV2_3:0')
i, o

(<tf.Tensor 'import/inputs:0' shape=(?,) dtype=string>,
 <tf.Tensor 'import/SelectV2_3:0' shape=(?, 256) dtype=int32>)

In [23]:
test_sess = tf.Session(graph = g)

In [24]:
import sentencepiece as spm
sp_model = spm.SentencePieceProcessor()
sp_model.Load(DEFAULT_SPM_PATH)

True

In [25]:
string1 = 'FORMAT TERBUKA. FORMAT TERBUKA IALAH SUATU FORMAT FAIL UNTUK TUJUAN MENYIMPAN DATA DIGITAL, DI MANA FORMAT INI DITAKRIFKAN BERDASARKAN SPESIFIKASI YANG DITERBITKAN DAN DIKENDALIKAN PERTUBUHAN PIAWAIAN , SERTA BOLEH DIGUNA PAKAI KHALAYAK RAMAI .'
string2 = 'Husein ska mkn ayam dkat kampng Jawa'

In [27]:
strings = [string1, string2]
[f'kes benar: {s}' for s in strings]

['kes benar: FORMAT TERBUKA. FORMAT TERBUKA IALAH SUATU FORMAT FAIL UNTUK TUJUAN MENYIMPAN DATA DIGITAL, DI MANA FORMAT INI DITAKRIFKAN BERDASARKAN SPESIFIKASI YANG DITERBITKAN DAN DIKENDALIKAN PERTUBUHAN PIAWAIAN , SERTA BOLEH DIGUNA PAKAI KHALAYAK RAMAI .',
 'kes benar: Husein ska mkn ayam dkat kampng Jawa']

In [28]:
%%time

o_ = test_sess.run(o, feed_dict = {i: [f'kes benar: {s}' for s in strings]})
o_.shape

CPU times: user 3.17 s, sys: 1.01 s, total: 4.18 s
Wall time: 1.53 s


(2, 256)

In [29]:
for k in range(len(o_)):
    print(k, sp_model.DecodeIds(o_[k].tolist()))

0 Format terbuka. Format terbuka ialah suatu format fail untuk tujuan menyimpan data digital, di mana format ini ditakrifkan berdasarkan spesifikasi yang diterbitkan dan dikendalikan pertubuhan piawaian, serta boleh diguna pakai khalayak ramai.
1 Husein ska mkn ayam dkat kampng Jawa.


In [30]:
from tensorflow.tools.graph_transforms import TransformGraph

In [31]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(minimum_size=1536000)',
             #'quantize_weights(fallback_min=-10240, fallback_max=10240)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [32]:
pb = 'super-super-tiny-true-case/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
    
transformed_graph_def = TransformGraph(input_graph_def, 
       ['inputs'],
       ['SelectV2_3'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [34]:
g = load_graph('super-super-tiny-true-case/frozen_model.pb.quantized')
i = g.get_tensor_by_name('import/inputs:0')
o = g.get_tensor_by_name('import/SelectV2_3:0')
i, o

(<tf.Tensor 'import/inputs:0' shape=(?,) dtype=string>,
 <tf.Tensor 'import/SelectV2_3:0' shape=(?, 256) dtype=int32>)

In [35]:
test_sess = tf.InteractiveSession(graph = g)

In [36]:
file = 'super-super-tiny-true-case/frozen_model.pb.quantized'
outPutname = 'true-case/super-super-tiny-t5-quantized/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [37]:
file = 'super-super-tiny-true-case/frozen_model.pb'
outPutname = 'true-case/super-super-tiny-t5/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)